In [1]:
from siamese_dataloader import Siamese_dataset, Siamese_dataloader
from torch import nn as nn
from torch import device, cuda, no_grad, manual_seed, cat
import torch.optim as optim
from torch.utils.data import Subset
from tqdm import tqdm
import numpy as np
import torch

RuntimeError: operator torchvision::nms does not exist

In [2]:
SEED = 2024

root="../train_xml"
# Raw data
subset = np.logical_not(np.load("data/missing_file_names_mask.npy")) # Makes losses adapt to the selected mouse_records
losses = np.load("data/resnet50_losses_final_weights.npy")[subset]
mouse_records = np.load("data/mouse_record_interpolated.npy")

# Clean data aka reasonable estimated time
estimate_times=np.load('data/sample_estimate_times.npy')
clean_subset = estimate_times[subset] < 3000
clean_subset = np.logical_and(clean_subset, estimate_times[subset] > 0)

losses = losses[clean_subset]
mouse_records = mouse_records[clean_subset]

# Train and test split
assert len(losses) == len(mouse_records)
indices = list(np.arange(len(losses)))
indices_randomized = np.random.choice(indices, int(len(indices)), replace=False)
train_indices = indices_randomized[:int(0.8*len(indices))]
test_indices = indices_randomized[int(0.8*len(indices)):]

losses_train = losses[train_indices]
mouse_records_train = mouse_records[train_indices]
dataset_train = Siamese_dataset(losses_train, mouse_records_train, seed=SEED)
dataloader_train = Siamese_dataloader(dataset_train, batch_size=4096, num_workers=8, shuffle=True).run()

losses_test = losses[test_indices]
mouse_records_test = mouse_records[test_indices]
dataset_test = Siamese_dataset(losses_test, mouse_records_test, seed=SEED)
dataloader_test = Siamese_dataloader(dataset_test, batch_size=4096, num_workers=8, shuffle=True).run()

len(dataset_train), len(dataset_test)

(872249, 218063)

In [3]:
def train_one_epoch(train_loader, net, criterion, optimizer, epoch):
    running_loss = 0.0
    for _, (inputs_0, inputs_1, labels) in enumerate(train_loader, 0):
        inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs_0, inputs_1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss

In [4]:
class Siamese_network(nn.Module):
    # It should predict 0, 1.
    # 0 if 1st sample has higher loss
    # 1 if 2nd sample has higher loss
    def __init__(self):
        super().__init__()

        # input is 30x3 dim and output is 1 dim
        self.shared_features = nn.Sequential(
            nn.Linear(60, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
        )

        self.classification_head = nn.Sequential(
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16, 1),
        )

    def forward(self, x, y):
        x = self.shared_features(x)
        y = self.shared_features(y)

        return self.classification_head(x * y)


In [5]:
cuda.is_available()

False

In [26]:
device = 'cuda:0' if cuda.is_available() else 'cpu'
print(f"Using device: {device}")

net = Siamese_network()
print(f"The number of parameters is {sum(p.numel() for p in net.parameters())}")
net.to(device)

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.8)

print("Training")
history = []

with no_grad():
    loss = 0
    accuracy = 0
    predictions = []
    all_labels = []
    for _, (inputs_0, inputs_1, labels) in enumerate(dataloader_test, 0):
        inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)

        outputs = net(inputs_0, inputs_1)
        predictions.append(outputs)
        all_labels.append(labels)
        loss += criterion(outputs, labels).item()

    predictions = cat(predictions)
    all_labels = cat(all_labels)
preds = predictions<0
accuracy = (preds == all_labels).sum().item() / len(predictions)
print(f"Initial test accuracy is {accuracy} and loss is {loss}")

Using device: cpu
The number of parameters is 165921
Training
Initial test accuracy is 0.5002269986196649 and loss is 38.64784795045853


In [27]:
device

'cpu'

In [29]:
EPOCHS = 50
for _, epoch in enumerate(tqdm(range(EPOCHS))):
    e_loss = train_one_epoch(dataloader_train, net, criterion, optimizer, epoch)
    print(f"Epoch: {epoch}/{EPOCHS} | Loss : {e_loss}")
    history.append(e_loss)

with no_grad():
    loss = 0
    accuracy = 0
    predictions = []
    all_labels = []
    for _, (inputs_0, inputs_1, labels) in enumerate(dataloader_train, 0):
        inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)
        outputs = net(inputs_0, inputs_1)
        predictions.append(outputs)
        all_labels.append(labels)
        loss += criterion(outputs, labels).item()

    predictions = cat(predictions)
    all_labels = cat(all_labels)
    preds = predictions<0
    train_accuracy = (preds == all_labels).sum().item() / len(predictions)

with no_grad():
    loss = 0
    accuracy = 0
    predictions = []
    all_labels = []
    for _, (inputs_0, inputs_1, labels) in enumerate(dataloader_test, 0):
        inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)
        outputs = net(inputs_0, inputs_1)
        predictions.append(outputs)
        all_labels.append(labels)
        loss += criterion(outputs, labels).item()

    predictions = cat(predictions)
    all_labels = cat(all_labels)
    preds = predictions<0
    accuracy = (preds == all_labels).sum().item() / len(predictions)
print(f"Finished Training, final test accuracy is {accuracy} and loss is {loss}, final training accuracy is {train_accuracy}")


  2%|▏         | 1/50 [00:37<30:30, 37.35s/it]

Epoch: 0/50 | Loss : 147.65708720684052


  4%|▍         | 2/50 [01:12<28:37, 35.78s/it]

Epoch: 1/50 | Loss : 147.6504823565483


  6%|▌         | 3/50 [01:45<27:11, 34.71s/it]

...

 98%|█████████▊| 49/50 [27:23<00:31, 31.70s/it]

Epoch: 48/50 | Loss : 147.64277404546738


100%|██████████| 50/50 [27:52<00:00, 33.46s/it]

Epoch: 49/50 | Loss : 147.6435136795044


Finished Training, final test accuracy is 0.5009744890238143 and loss is 37.4307045340538, final training accuracy is 0.49955230673809886
